In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Conv1D, Dropout
from sklearn.model_selection import train_test_split
from skimage.transform import resize
import librosa
from tensorflow.keras.layers import Input, LSTM, Dense

data_dir = '/Users/fahad/Desktop/sounds final  2'
classes = ['Civil', 'Police', 'Trafic', 'ambulance']


def load_and_preprocess_data(data_dir, classes, target_shape=(128, 128)):
    data = []
    labels = []

    for i, class_name in enumerate(classes):
        class_dir = os.path.join(data_dir, class_name)
        for filename in os.listdir(class_dir):
            if filename.endswith('.wav'):
                file_path = os.path.join(class_dir, filename)
                audio_data, sample_rate = librosa.load(file_path, sr=None)

                mel_spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate)
                mel_spectrogram_resized = resize(mel_spectrogram, target_shape)
                data.append(mel_spectrogram_resized)
                labels.append(i)

    return np.array(data), np.array(labels)


data, labels = load_and_preprocess_data(data_dir, classes)
labels = to_categorical(labels, num_classes=len(classes))
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)



# Define the LSTM model
model = Sequential()
model.add(Dense(256, input_shape=(40,), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(192, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.5))
#model.add(Dense(len(labels), activation='softmax'))

input_shape = X_train[0].shape
input_layer = Input(shape=input_shape)
x = LSTM(64)(input_layer)
output_layer = Dense(len(classes), activation='softmax')(x)
model = Model(inputs=input_layer, outputs=output_layer)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, validation_split=0.1)


test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

Epoch 1/100
4/4 [==============================] - 4s 262ms/step - loss: 1.3857 - accuracy: 0.2269 - val_loss: 1.3946 - val_accuracy: 0.1429
Epoch 2/100
4/4 [==============================] - 0s 75ms/step - loss: 1.3825 - accuracy: 0.2773 - val_loss: 1.4035 - val_accuracy: 0.0714
Epoch 3/100
4/4 [==============================] - 0s 66ms/step - loss: 1.3803 - accuracy: 0.2857 - val_loss: 1.4174 - val_accuracy: 0.1429
Epoch 4/100
4/4 [==============================] - 0s 65ms/step - loss: 1.3760 - accuracy: 0.3277 - val_loss: 1.4331 - val_accuracy: 0.1429
Epoch 5/100
4/4 [==============================] - 0s 65ms/step - loss: 1.3674 - accuracy: 0.2857 - val_loss: 1.4664 - val_accuracy: 0.1429
Epoch 6/100
4/4 [==============================] - 0s 69ms/step - loss: 1.3578 - accuracy: 0.2857 - val_loss: 1.4782 - val_accuracy: 0.1429
Epoch 7/100
4/4 [==============================] - 0s 69ms/step - loss: 1.3491 - accuracy: 0.2857 - val_loss: 1.4794 - val_accuracy: 0.1429
Epoch 8/100
4/4 [==

In [ ]:
import os
import numpy as np
import librosa
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from skimage.transform import resize
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping , ModelCheckpoint
from datetime import datetime

data_dir = '/Users/irk2w/Desktop/T5/sounds_final '
classes = ['Civil', 'Police', 'Trafic', 'ambulance']

def extract_features(file_path, target_shape=(128, 128)):
    audio_data, sample_rate = librosa.load(file_path, sr=None)

    # Data Augmentation
    pitch_shifted = librosa.effects.pitch_shift(audio_data, sr=sample_rate, n_steps=4)
    time_stretched = librosa.effects.time_stretch(audio_data, rate=1.5)

    # Original Features
    mel_spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate)
    # Augmented Features
    mel_spectrogram_pitch = librosa.feature.melspectrogram(y=pitch_shifted, sr=sample_rate)
    mel_spectrogram_stretch = librosa.feature.melspectrogram(y=time_stretched, sr=sample_rate)

    # Resizing
    mel_spectrogram_resized = resize(mel_spectrogram, target_shape)
    mel_spectrogram_pitch_resized = resize(mel_spectrogram_pitch, target_shape)
    mel_spectrogram_stretch_resized = resize(mel_spectrogram_stretch, target_shape)

    return mel_spectrogram_resized, mel_spectrogram_pitch_resized, mel_spectrogram_stretch_resized

def load_and_preprocess_data(data_dir, classes, target_shape=(128, 128)):
    data = []
    labels = []

    for i, class_name in enumerate(classes):
        class_dir = os.path.join(data_dir, class_name)
        for filename in os.listdir(class_dir):
            if filename.endswith('.wav'):
                file_path = os.path.join(class_dir, filename)
                original, pitch, stretch = extract_features(file_path, target_shape)
                # Append original features
                data.append(original)
                labels.append(i)
                # Append augmented features
                data.append(pitch)
                labels.append(i)
                data.append(stretch)
                labels.append(i)

    return np.array(data), np.array(labels)

data, labels = load_and_preprocess_data(data_dir, classes)
labels = to_categorical(labels, num_classes=len(classes))
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

model = Sequential()
model.add(Dense(256, input_shape=(128, 128), activation='relu'))  # Adjust the input_shape according to your data
model.add(Dropout(0.5))
model.add(Dense(192, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
#model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(LSTM(64))
model.add(Dense(len(classes), activation='softmax'))



# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

# Display model architecture summary
model.summary()

# Calculate pre-training accuracy
score = model.evaluate(X_test, y_test, verbose=0)
accuracy = 100*score[1]

# Print pre-training accuracy
print("Pre-training accuracy: %.4f%%" % accuracy)

# Train the model
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_mlp.hdf5',
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(X_train, y_train, batch_size=256, epochs=400, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)

duration = datetime.now() - start
print("Training completed in time: ", duration)

# Evaluating the model on the training and testing set
score = model.evaluate(X_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(X_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

In [3]:
model.save('911_class.h5')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [3]:
from tensorflow.keras.models import load_model

def predict_audio_class(file_path, model, classes):
    # Assuming the extract_features function is defined as above
    features = extract_features(file_path)[0]  # Use the original features for prediction
    features = np.expand_dims(features, axis=0)  # Reshaping to match model input

    # Make the prediction
    predictions = model.predict(features)
    predicted_class = np.argmax(predictions, axis=1)
    return classes[predicted_class[0]]

# Usage
model = load_model("Desktop/T5/911_model_don't_touch/911_class_test.h5")  # Load your model
audio_file_path = '/Users/irk2w/Downloads/1.wav'
predicted_class_name = predict_audio_class(audio_file_path, model, classes)
print(f"The model predicts that the audio file is a {predicted_class_name} sound.")

1/1 [==============================] - 0s 256ms/step
The model predicts that the audio file is a Police sound.


In [ ]:
pip install gTTS

In [ ]:
from gtts import gTTS
gtts = gTTS('مرحبًا، أود الإبلاغ عن رصد حقيبة مشبوهة تُترك بمفردها في محطة القطار', lang='ar')
gtts.save('police-10.mp3')

In [ ]:
input_shape = X_train[0].shape
input_layer = Input(shape=input_shape)
x = LSTM(64)(input_layer)
output_layer = Dense(len(classes), activation='softmax')(x)
model = Model(inputs=input_layer, outputs=output_layer)


In [3]:
from tensorflow.keras.layers import Bidirectional, Dropout
from tensorflow.keras.regularizers import l2

# ... Your data loading and preprocessing code ...

# Normalization (example using global mean and std)
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)
X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

# Improved model architecture
input_layer = Input(shape=input_shape)

# Consider adding Conv2D layers here if you want to use CNN features

# Using bidirectional LSTM and adding dropout
x = Bidirectional(LSTM(64, return_sequences=True, dropout=0.5))(input_layer)
x = Flatten()(x)  # Flatten needed if return_sequences=True
x = Dense(64, activation='relu', kernel_regularizer=l2(0.001))(x)  # Regularized dense layer
x = Dropout(0.5)(x)  # Dropout for regularization

output_layer = Dense(len(classes), activation='softmax')(x)

model = Model(inputs=input_layer, outputs=output_layer)

# Compile and fit the model with some form of learning rate scheduler or reduction on plateau
from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, validation_split=0.1, callbacks=[reduce_lr])

Epoch 1/10
4/4 [==============================] - 6s 442ms/step - loss: 1.6455 - accuracy: 0.2353 - val_loss: 2.0695 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 2/10
4/4 [==============================] - 1s 164ms/step - loss: 0.8984 - accuracy: 0.7143 - val_loss: 2.0566 - val_accuracy: 0.2143 - lr: 0.0010
Epoch 3/10
4/4 [==============================] - 1s 165ms/step - loss: 0.6367 - accuracy: 0.7899 - val_loss: 2.0225 - val_accuracy: 0.2143 - lr: 0.0010
Epoch 4/10
4/4 [==============================] - 1s 154ms/step - loss: 0.5065 - accuracy: 0.9076 - val_loss: 2.2557 - val_accuracy: 0.2857 - lr: 0.0010
Epoch 5/10
4/4 [==============================] - 1s 167ms/step - loss: 0.4022 - accuracy: 0.9328 - val_loss: 2.3901 - val_accuracy: 0.2857 - lr: 0.0010
Epoch 6/10
4/4 [==============================] - 1s 170ms/step - loss: 0.3839 - accuracy: 0.9160 - val_loss: 2.6047 - val_accuracy: 0.2857 - lr: 0.0010
Epoch 7/10
4/4 [==============================] - 1s 165ms/step - loss: 0.2956

In [7]:
import os
import numpy as np
import librosa
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional , Input
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping , ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from skimage.transform import resize
from datetime import datetime
from tensorflow.keras.models import Model

# Define a function to extract MFCC features from audio files
def extract_features(file_name):
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccs_processed = np.mean(mfccs.T, axis=0)
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None
    return mfccs_processed

# Define the dataset directory and the labels
data_dir = '/content/drive/MyDrive/New folder/T5/sounds final'
classes = ['Civil', 'Police', 'Trafic', 'ambulance']


def load_and_preprocess_data(data_dir, classes, target_shape=(128, 128)):
    data = []
    labels = []

    for i, class_name in enumerate(classes):
        class_dir = os.path.join(data_dir, class_name)
        for filename in os.listdir(class_dir):
            if filename.endswith('.wav'):
                file_path = os.path.join(class_dir, filename)
                audio_data, sample_rate = librosa.load(file_path, sr=None)

                mel_spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate)
                mel_spectrogram_resized = resize(mel_spectrogram, target_shape)
                data.append(mel_spectrogram_resized)
                labels.append(i)

    return np.array(data), np.array(labels)

# Encode the labels to integers
data, labels = load_and_preprocess_data(data_dir, classes)
labels = to_categorical(labels, num_classes=len(classes))
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)




# Define the LSTM model
model = Sequential()
model.add(Dense(256, input_shape=(128, 128), activation='relu'))  # Adjust the input_shape according to your data
model.add(Dropout(0.5))
model.add(Dense(192, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
#model.add(LSTM(64))
#model.add(Dense(len(classes), activation='softmax'))

input_shape = X_train[0].shape
input_layer = Input(shape=input_shape)
x = LSTM(64)(input_layer)
output_layer = Dense(len(classes), activation='softmax')(x)
model = Model(inputs=input_layer, outputs=output_layer)


# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

# Display model architecture summary
model.summary()

# Calculate pre-training accuracy
score = model.evaluate(X_test, y_test, verbose=0)
accuracy = 100*score[1]

# Print pre-training accuracy
print("Pre-training accuracy: %.4f%%" % accuracy)

# Train the model
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_mlp.hdf5',
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(X_train, y_train, batch_size=256, epochs=100, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)

duration = datetime.now() - start
print("Training completed in time: ", duration)

# Evaluating the model on the training and testing set
score = model.evaluate(X_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(X_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128)]        0         
                                                                 
 lstm_4 (LSTM)               (None, 64)                49408     
                                                                 
 dense_22 (Dense)            (None, 4)                 260       
                                                                 
Total params: 49668 (194.02 KB)
Trainable params: 49668 (194.02 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Pre-training accuracy: 14.7059%
Epoch 1/100
1/1 [==============================] - ETA: 0s - loss: 1.3862 - accuracy: 0.2556
Epoch 1: val_loss improved from inf to 1.38667, saving model to saved_models/weights.best.basic_mlp.hdf5
1/1 [==============================] - 2s 2s/step - loss: